In [16]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.tracers.stdout import ConsoleCallbackHandler

In [17]:
os.environ['OPENAI_API_KEY'] = "여기에 API 키를 넣어주세요"

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1, # 창의성 (0.0 ~ 2.0)
    max_tokens=2048, # 최대 토큰수
    model_name="gpt-3.5-turbo", # 모델명
)

# 질의내용
question = "세종대왕이 누구인지 설명해주세요"

# 질의
result = llm.invoke(question)
print(result.content)

세종대왕은 조선시대 4대 왕 중 한 명으로, 조선시대의 제4대 군주로 알려져 있습니다. 그의 본명은 이도이며, 세종대왕은 1418년에서 1450년 사이에 통치했습니다. 세종대왕은 조선시대의 중요한 업적을 남긴 왕으로, 특히 한글을 창제하고 과학기술, 문화, 예술 분야에서 많은 발전을 이루었습니다. 또한 세종대왕은 국가의 행정체계를 개혁하고 사회적 불평등을 해소하기 위해 노력했습니다. 그의 통치는 조선시대의 전성기를 이루는 데 큰 역할을 했습니다.


In [3]:
# 질문 템플릿 형식 정의
template = "{who}가 누구인지 설명해주세요"

# 템플릿 완성
prompt = PromptTemplate(
    template=template, input_variables=['who']
)
print(prompt)

input_variables=['who'] input_types={} partial_variables={} template='{who}가 누구인지 설명해주세요'


In [4]:
print(prompt.format(who="오바마"))

오바마가 누구인지 설명해주세요


In [5]:
# 연결된 체인 생성
llm_chain = prompt | llm
result = llm_chain.invoke({"who":"이순신 장군"})
print(result.content)

이순신 장군은 조선시대의 무신이자 해군 장군으로, 16세기 말부터 17세기 초까지 활약한 역사적 인물입니다. 이순신은 조선의 해군을 세워 조선을 외적으로부터 방어하고 일본의 침략을 막는 데 큰 역할을 했습니다. 특히, 명량 해전에서 일본의 강력한 함대를 물리치는 데 성공하여 역사상 가장 위대한 해전 중 하나로 꼽히고 있습니다. 이순신은 뛰어난 전략가로서 뿐만 아니라 용감한 전투력으로 많은 사람들에게 존경받는 인물입니다. 현재까지도 그의 업적은 많은 사람들에게 기억되고 있습니다.


In [7]:
result = llm_chain.invoke({"who":"이순신 장군"},
                          config={"callbacks": [ConsoleCallbackHandler()]})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "who": "이순신 장군"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "who": "이순신 장군"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이순신 장군가 누구인지 설명해주세요"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [3.29s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이순신 장군은 조선시대의 무신이자 해군 장군으로, 16세기 말부터 17세기 초까지 활약한 역사적 인물입니다. 이순신은 조선 왕조 말기에 일본의 침략에 맞서 싸우며 일본의 대외 정책을 저지하고 조선의 해군을 강화하는 데 큰 역할을 했습니다. 특히, 이순신은 거북선을 개발하여 일본의 강력한 함대를 상대로 승리를 거두며 전쟁에서 중요한 역할을 했습니다. 이순신은 조선의 무신으로 인정받아 현재까지도 그의 업적과 뛰어난 전략가로서의 능력이 인정받고 있습니다.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "mess

In [12]:
# 프롬프트 템플릿 생성
template = """아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 위키북스봇입니다. 대화 문맥을 바탕으로 친절한 답변을 진행하세요.

Current Conversation:
{history}

Human: {input}
AI:"""

prompt = PromptTemplate(
    template=template, input_variables=['history', 'input']
)

In [13]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
chain = prompt | llm

In [14]:
store = {}
session_id = "test"

In [15]:
if session_id not in store:
    store[session_id] = ChatMessageHistory()
session_history = store[session_id]

In [19]:
with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: session_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [20]:
result = with_message_history.invoke(
    {"input": "당신은 어디에서 만들었습니까?"},
    config={"configurable": {"session_id": "test"}},
)
print(result.content)

안녕하세요! 저는 위키북스봇이라고 합니다. 저는 한국에서 만들어졌어요. 어떻게 도와드릴까요?


In [21]:
result = with_message_history.invoke(
    {"input": "푸른 바다를 주제로 감성적이고 짧은 시를 하나 지어주세요"},
    config={"configurable": {"session_id": "test"}},
)
print(result.content)

파란 파도가 춤추는 푸른 바다,
마음을 가라앉히는 향기로운 소나무.
바람이 부는 소리에 귀를 기울이면,
행복한 순간에 머무는 시간.


In [22]:
result = with_message_history.invoke(
    {"input": "석양을 주제로도 해줘"},
    config={"configurable": {"session_id": "test"}},
)
print(result.content)

붉게 물든 하늘을 바라보며,
얼마나 아름다운 석양인지를 느끼며,
하루의 끝을 달래는 따뜻한 빛,
잊을 수 없는 순간, 세상의 평화.


In [23]:
print(store)

{'test': InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 어디에서 만들었습니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 저는 위키북스봇이라고 합니다. 저는 한국에서 만들어졌어요. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-44b73423-0321-4b24-a121-8ec5c2484c94-0', usage_metadata={'input_tokens': 92, 'output_tokens': 56, 'total_tokens': 148}), HumanMessage(content='푸른 바다를 주제로 감성적이고 짧은 시를 하나 지어주세요', additional_kwargs={}, response_metadata={}), AIMessage(content='파란 파도가 춤추는 푸른 바다,\n마음을 가라앉히는 향기로운 소나무.\n바람이 부는 소리에 귀를 기울이면,\n행복한 순간에 머무는 시간.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 328, 'total_tokens': 409, 'completion_tokens_detail